# 🏗️ Vue d'ensemble de l'architecture DoctorPy

Ce notebook présente l'architecture microservices de DoctorPy et montre comment interagir avec les différents services.

## 🎯 Objectifs
- Comprendre l'architecture microservices
- Tester les APIs de chaque service
- Voir l'intégration entre services

## 📊 Architecture des services

DoctorPy utilise 5 microservices principaux :

| Service | Port | Responsabilité |
|---------|------|----------------|
| Auth | 8001 | Authentification et profils |
| RAG | 8002 | Assistant IA et génération |
| Analytics | 8003 | Métriques et insights |
| Quest | 8004 | Gamification et quêtes |
| Notification | 8005 | Notifications multi-canal |

In [ ]:
import requests
import json
from IPython.display import display, JSON

# Configuration des services
services = {
    "auth": "http://localhost:8001",
    "rag": "http://localhost:8002", 
    "analytics": "http://localhost:8003",
    "quest": "http://localhost:8004",
    "notification": "http://localhost:8005"
}

print("🌟 Configuration des services DoctorPy")
for name, url in services.items():
    print(f"  • {name.upper()}: {url}")

## 🔍 Test de santé des services

Vérifions que tous les services sont opérationnels :

In [ ]:
def check_service_health(name, base_url):
    """Vérifie la santé d'un service"""
    try:
        response = requests.get(f"{base_url}/health", timeout=5)
        if response.status_code == 200:
            data = response.json()
            print(f"✅ {name.upper()}: {data.get('status', 'unknown')}")
            return data
        else:
            print(f"⚠️ {name.upper()}: Status {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ {name.upper()}: Non accessible ({str(e)[:50]}...)")
        return None

print("🏥 Vérification de la santé des services:")
health_results = {}
for name, url in services.items():
    health_results[name] = check_service_health(name, url)

## 🤖 Test du service RAG

Testons l'assistant IA avec quelques questions Python :

In [ ]:
def test_rag_query(query, query_type="general"):
    """Teste une requête RAG"""
    try:
        response = requests.post(
            f"{services['rag']}/api/v1/rag/query",
            json={"query": query, "query_type": query_type}
        )
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Status {response.status_code}"}
    except Exception as e:
        return {"error": str(e)}

# Test de différentes requêtes
queries = [
    ("Comment créer une variable en Python ?", "code_help"),
    ("Explique-moi les boucles for", "tutorial"),
    ("Qu'est-ce qu'une fonction ?", "concept")
]

print("🤖 Test du service RAG:")
for query, query_type in queries:
    print(f"\n❓ Question: {query}")
    result = test_rag_query(query, query_type)
    if "error" not in result:
        print(f"💡 Réponse: {result.get('response', '')[:100]}...")
        print(f"📚 Sources: {', '.join(result.get('sources', []))}")
        print(f"🎯 Confiance: {result.get('confidence', 0)*100:.0f}%")
    else:
        print(f"❌ Erreur: {result['error']}")

## 🎮 Test du service Quest

Explorons le système de gamification :

In [ ]:
# Récupérer les quêtes disponibles
try:
    response = requests.get(f"{services['quest']}/api/v1/quests")
    if response.status_code == 200:
        quests_data = response.json()
        quests = quests_data.get('quests', [])
        
        print(f"🎯 Quêtes disponibles ({len(quests)}):")
        for quest in quests:
            print(f"  • {quest['title']} ({quest['difficulty']})")
            print(f"    Points: {quest['points']} | Temps estimé: {quest['estimated_time']}")
            print(f"    {quest['description']}")
            print()
        
        # Afficher sous forme de tableau
        import pandas as pd
        df = pd.DataFrame(quests)
        display(df[['title', 'difficulty', 'points', 'estimated_time', 'questions_count']])
    else:
        print(f"❌ Erreur récupération quêtes: {response.status_code}")
except Exception as e:
    print(f"❌ Erreur: {e}")

## 📊 Test du service Analytics

Regardons les métriques du système :

In [ ]:
# Vue d'ensemble des analytics
try:
    response = requests.get(f"{services['analytics']}/api/v1/analytics/overview")
    if response.status_code == 200:
        overview = response.json()
        summary = overview.get('summary', {})
        
        print("📈 Vue d'ensemble Analytics:")
        print(f"  • Utilisateurs totaux: {summary.get('total_users', 0)}")
        print(f"  • Utilisateurs actifs aujourd'hui: {summary.get('active_users_today', 0)}")
        print(f"  • Taux de complétion des quêtes: {summary.get('quest_completion_rate', 0)*100:.1f}%")
        print(f"  • Durée moyenne de session: {summary.get('avg_session_duration', 'N/A')}")
        
        # Affichage structuré
        display(JSON(overview))
    else:
        print(f"❌ Erreur analytics: {response.status_code}")
except Exception as e:
    print(f"❌ Erreur: {e}")

## 🔔 Test du service Notification

Testons le système de notifications :

In [ ]:
# Test d'envoi de notification
notification_data = {
    "user_id": "notebook_user",
    "message": "🎉 Test depuis le notebook Jupyter ! Architecture microservices opérationnelle.",
    "type": "info",
    "channels": ["in_app", "email"]
}

try:
    response = requests.post(
        f"{services['notification']}/api/v1/notifications/send",
        json=notification_data
    )
    if response.status_code == 200:
        result = response.json()
        print("📨 Notification envoyée avec succès:")
        print(f"  • ID: {result.get('notification_id')}")
        print(f"  • Statut: {result.get('status')}")
        print(f"  • Canaux: {[r['channel'] for r in result.get('channels_results', [])]}")
        
        display(JSON(result))
    else:
        print(f"❌ Erreur notification: {response.status_code}")
except Exception as e:
    print(f"❌ Erreur: {e}")

## 🎉 Conclusion

Ce notebook a démontré :

✅ **Architecture microservices opérationnelle**  
✅ **5 services indépendants et communicants**  
✅ **APIs REST complètes et documentées**  
✅ **Intégration parfaite entre services**  

L'écosystème DoctorPy est prêt pour le développement et l'extension !